**2020-04-10**

1. 模型 resnet18_ssd
    - 在3种分辨率上预测，使用单独的ssd head
    - 试验了多种优化器，在tiny_ds上训练了5k多个iteration，最优结果的分类正确率不足0.75

2. 模型 resnet18_1ssd
    - 主要改变在于 **1**ssd，即3个分辨率的预测共用一个 ssd head
    - 想法是：
        - 每个分辨率单独一个ssd head，则每个 ssd head 只能在相应分辨率的目标上被训练到，训练量就较少。
        - 而如果共用一个 ssd head，则 ssd head 的训练量就多了，这有助于 ssd head 的训练
        - 但代价时要引入3个 neck block，它们也是要训练的，但我认为它们较简单，应该可以比较容易训练好。

3. 目标检测模型的**数据集内预训练**
    - 从craft训练目标检测任务，可能不好训练，因为目标检测涉及到 分类，置信，定位 多个目标
    - 提出一种猜想：数据集内预训练
        - 以多分类任务预训练
        - 按3种分辨率分别预训练
    - 生成预训练用的数据集
        - 一次仅考虑一个尺度范围内的目标
        - 仅保留该类目标，其它尺度的目标被抹掉（涂白或背景填充）
    - 预训练
        - 每次仅考虑ssd head中于一个尺度范围的目标的**分类相关**的输出的损失函数
        - 依次对各个尺度范围单独训练，从小至大训练
        - 已经训练好的层保持freeze，不参与后续尺度的训练
        - ssd head中分类相关的参数仅参与第一尺度范围的训练，在后续训练中锁定
    - 训练
        - 预训练完后，以预训练的参数为初始，进行正常的目标检测训练

**2020-04-12**

1. 训练记录：
    - 数据集：tiny_ds_20200331
    - 模型：resnet18_1ssd
    - 备注：pConf 和 nConf 使用BCE loss
    - 优化器：
        - Adam，
        - LAMB，
        - Pre_LAMB，
        - UM
    - log：./run_log/20200412/

**2020-04-13**

1. 训练记录：
    - 数据集：ds_20200227
    - 模型：resnet18_1ssd, partial init to pretrained resnet18
    - 损失函数：与“2020-04-12/1.训练记录”一样
    - bs：64
    - 优化器：
        - Adam
        - LAMB（发散，中止）
    - log：./run_log/20200413
    - 简述：提升明显

2. 在 fastai 中如何使用 discrimitive lr？
    - 在 fastai 中，可以两种方式指定 lr，一个是在 fit 函数中，一个是在 scheduler 中。
    - 要使用 discrimitive lr，对这两种方式来说是一样的。
    - 首先要设置 learn.layer_groups，要包括多个 layer groups
    - 然后再设置 lr 时，将 lr 置为一个 numpy array，其长度与 layer_groups 一致。

3. fastai 的 layer_groups
    - fastai 的 learner 的 layer_groups 相当于 pytorch 中的 optimizer 的 param_groups
    - 但是当你设置了 learner 的 layer_groups 后，在训练时会发现 optimizer 中 param_groups 的个数是 layer_groups 的 2 倍。
    - 这是因为，fastai 把一个 layer_group 内的参数又分成了两部分，第一部分是可以被weight decay的，第二部分是不可以被weight decay的。见 split_no_wd_params 函数，它执行这个划分。
    - 什么参数可以被 weight decay，什么参数不可以？
        - weight decay 的作用是降低模型的复杂度。
        - 模型的参数中，有些是与模型复杂度有关的，例如卷积层和全连接层的weight，这些参数被weight decay是有意义的；
        - 有些是与模型的复杂度无关的，例如卷积层和全连接层的bias，batchnorm层的weight和bias，这些参数被weight decay没有意义。

4. fastai 的 optimWraper
    - fastai 的 optimWraper 是对 optimizer 的一个包装，它自动执行一些与optimizer相关的常见或者说标准操作
    - 它对optimizer的实现做了一些限制，例如你的optimizer的state里必须有betas，且betas必须是一个两元素的tuple
    - 它内置的 true weight decay，如果你启用了该功能，则 optimWraper 就会自动执行 weight decay，如果你不清楚这一点，你在optimizer的step中也做了weight decay，那么就相当于执行了两边weight decay，就会有问题。
    - 我认为 optimWraper 是一个很不好的设计，它隐形地对你实现 optimizer 时增加了一些限制，甚至挖了一些坑。属于过度服务。

5. 对 transfer-learning, finetune 和 discriminative lr 的理解
    - transfer learning 是利用已经训练好的模型的backbone，在其基础上添加新的head，适应新任务
    - transfer learning 可以对整个模型进行训练，试验过这种方法，效果并不差。
    - transfer learning 的另一种方法是，先冻结预训练的backbone，让新添加的部分训练一段时间；然后解冻backbone，对整个模型进行训练。这就是finetune.
    - finetune 时，需要用到 discriminative lr: 设置两个 layer_group，一个包含backbone，lr设为0；另一个包含新添加的部分，lr正常设置。在这样的设置下训练一段时间后，等新添加的部分基本收敛了，再将两个group的lr都设为非零，开始训练整个模型。
    - discriminative lr 的用途：
        - 上面提到，discriminative lr 在 finetune 中很有用，在其它情况下呢？
        - 我觉得只应以一种方式使用 discriminative lr：两个group，一个group的lr设为0，另一个的lr设为非零
        - 虽然 discriminative lr 的使用可以很灵活，例如分作多个group，且各个group的lr都是不同的非零值。

**2020-04-15**

1. 训练记录
    - 在“2020-4-13，训练记录”之上的变化
    - 优化器：Adam
    - 设置anneal阈值较高，worseN_thres=10
    - 第一次run是先finetune一个stage后全部训练，结果不好
    - 第二次run无finetune，但是直到200epoch结束也没有anneal，最终结果与“2020-4-13，训练记录”很接近。（计划：以更长的epoch训练）
    - log：./run_log/20200415

2. 训练计划
    - 设置 conf_th = 1
    - 使用clas_weights, 令max_imblance=5
    - 两种优化器：Stable_Adam和Adam

3. To Do (not done):
    - Stable_Adam 使用 exp avg 的 ||theta||，而非当前 ||theta||
    - 并且其也使用beta1，即与m一致
    - 预期这样更改可以使更新步长更稳定

4. To Do (not done):
    - 自适应weight decay
    - 实验发现，使用Stable_Adam优化器时，weight scale增长的更快，这不是一个好现象
    - 在文章里(https://blog.janestreet.com/l2-regularization-and-batch-norm/)写道，使用batchnorm而无weight decay时，weight scale会随时间有变大的趋势，而设置合适的weight decay可以抑制这个趋势，使weight scale保持稳定。
    - 这个weight decay应该设多大恰好可以抵消这个趋势？不好把握。所以是否可以用“自适应weight decay”
    - 自适应wd：
        - 应用于后跟bn的卷积层，
        - 目标是使其weight scale保持稳定，
        - 方法：记录初始scale，在之后的时刻，完成更新后，将weight归一化到初始scale，即乘以 init_scale/current_scale

**2020-04-17**

1. 训练记录
    - 在“2020-4-15，1.训练记录”之上的变化:
        - conf_th: 0.5 -> 1
        - 使用 clas_weights，使 max_imblance=5
        - 对比两种优化器：Adam和Stable_Adam，都是全程无freeze
        - 设置最大500epoch，保证充分训练
    - 第一次run：
        - Adam
        - 在390个epoch结束
        - 最终效果显著提高，valid_loss为1.26
    - 第二次run：
        - Stable_Adam
        - 在38 epoch处手动结束
        - 开始时比Adam快，逐渐变慢，在epoch#30开始劣于Adam
        - update波动大
    - log:
        - ./run_log/20200417/

**2020-04-19**

1. 训练记录
    - 在“2020-04-17, 1.训练记录”之上的变化：
        - 对比两种优化器：Adam_AWD和Stable_Adam
        - 这两种优化器都是修改了的：由原来对参数按layer的整体scale，改为按chout的整体scale
    - 第一次run：
        - Adam_AWD
        - 在epoch#240左右结束
        - 最终效果与“2020-04-17, 1.训练记录，第一次run”基本一致，但是训练时间短了很多
        - 观察了conv.weight的整体scale全程较稳定
    - 第二次run：
        - Stable_Adam
        - 因为其它程序抢占cpu，导致该run在epoch#258异常结束
        - 直到epoch#258，未anneal，达到valid_loss约1.36
        - 从异常结束处继续训练，在epoch#120处结束，最终valid_loss达到了1.16
        - 观察了conv.weight的整体scale不断变大，达到了数百甚至上万的量级
    - log:
        - ./run_log/20200419/
        - run_1.csv仅记录了继续训练的部分

**2020-04-20**

1. 训练记录
    - 在“2020-04-19，1.训练记录”之上的变化
        - 修改了 resnet_ssd.flatten_grid_anchor 函数，由原来横向（y）先变化改为纵向（x）先变化，以与grid设置一致
        - 仅使用Stable_Adam优化器
    - run_0：
        - Stable_Adam
        - 在epoch#240结束
        - 进步非常明显，最优valid_loss为0.111，各项metrics几乎理想
    - 此次训练所作debug是关键，之前的训练都是有bug的
    - 直到训练结束未见valid_loss上升
        - 实际上即使在之前有bug的训练中也未见valid_loss上升
        - 这说明valid数据与train数据太相似了，实际上valid数据已经不能起到valid的作用了
        - 这也说明该符号检测任务的边界非常明确并且范围很小，我们的数据集已经遍历了所有数据分布。
        - 这是一种特殊情况
    - log：
        - ./run_log/20200220

**2020-04-24**

1. 训练记录
    - 在“2020-04-20,1.训练记录”之上的变化
        - 测试 Adam 和 Adam_AWD 优化器
    - run_0:
        - Adam
        - 在 epoch#165 结束，最好 valid_loss=0.117
        - 与 Stable_Adam 相比，结束更快，但 valid_loss 稍差
    - run_1：
        - Adam_AWD
        - 在 epoch#60 处 smooth_loss 出现 inf，导致提前结束
        - 未观察到其正常训练结果如何，但可以看出Adam_AWD容易出现loss异常大的值
    - log:
        - ./run_log/20200224

**2020-04-26**

1. 训练记录
    - 在“2020-04-24,1.训练记录”之上的变化
        - 测试 Adam_AWD 优化器
        - 对fastai库做了修改使smooth_loss不会出现inf
    - run_0：
        - Adam_AWD
        - 在epoch#201结束，最好 valid_loss=0.117
        - 与 Adam 结果接近，但结束更慢
    - log:
        - ./run_log/20200226

**2020-04-28**

1. 训练记录
    - 在“2020-04-26，1.训练记录”之上的变化
        - 从随机参数开始训练
    - run_0:
        - Adam
        - 在 epoch#295 结束，最优valid_loss=0.102
        - 与从预训练的resnet18开始训练相比，更慢，但结果稍好
    - run_1：
        - Stable_Adam
        - 在 epoch#262 结束，最优 valid_loss=0.107
        - 与从预训练的resnet18开始训练相比，更慢，但结果稍好
        - 与Adam相比，稍快，但结果稍差
    - log:
        - ./run_log/20200228

**2020-04-29**

1. 训练记录
    - 在“2020-04-28，1.训练记录”之上的变化
        - 使用class_weights
        - lambda_nConf=100